<a href="https://colab.research.google.com/github/Divyanshu451/Image/blob/master/image_captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from os import listdir
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model


Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
def extract_features(directory):
  model=VGG16()
  model.layers.pop()
  model=Model(inputs=model.inputs, outputs=model.layers[-1].output)
  print(model.summary())
  features=dict()
  for name in listdir(directory):
    filename=directory + '/' + name
    image=load_img(filename, target_size=(224,224))
    image=img_to_array(image)
    image=image.reshape((1,image.shape[0], image.shape[1], image.shape[2]))
    image=preprocess_input(image)
    feature= model.predict(image, verbose=0)
    image_id=name.split('.')[0]
    features[image_id]= feature
    print('>%s' % name)
  return features




In [ ]:
directory='/content/drive/My Drive/Flickr_Data/Flicker8k_Dataset'

In [ ]:
features=extract_features(directory)
print('Extracted Features: %d' % len(features))
dump(features, open('/content/drive/My Drive/Flickr_Data/features2.pkl','wb'))

Streaming output truncated to the last 5000 lines.
>3252457866_b86614064c.jpg
>3250695024_93e8ab7305.jpg
>3249125548_700d874380.jpg
>3251088971_f4471048e3.jpg
>3251460982_4578a568bb.jpg
>3249891874_6a090ef097.jpg
>3248752274_96740ed073.jpg
>3247693965_845b3b4349.jpg
>3255620561_7644747791.jpg
>3249062399_0dafe5e4f5.jpg
>3248352729_ab264b2222.jpg
>3254645823_a7c072481c.jpg
>3255732353_fbc487aefc.jpg
>3251976937_20625dc2b8.jpg
>3255737244_1f8948fc07.jpg
>3251648670_9339943ba2.jpg
>3247500085_c4f641aa84.jpg
>3248408149_41a8dd90d3.jpg
>3249891130_b241591e89.jpg
>3254817653_632e840423.jpg
>3251906388_c09d44340e.jpg
>3254640083_eb34b8edfe.jpg
>3256272547_5ae6c66293 (3).jpg
>3257107194_f235c8f7ab (1).jpg
>3256275785_9c3af57576 (3).jpg
>3258874419_23fec1bdc1.jpg
>326028454_fb396167e6.jpg
>3258472448_75cfab5e6f.jpg
>3259694057_fae7484b0a.jpg
>3259110412_9908c45144.jpg
>3259666643_ae49524c81.jpg
>3256272547_5ae6c66293 (2).jpg
>3256274183_4eab3b2322 (3).jpg
>3256275785_9c3af57576 (2).jpg
>3259992

In [ ]:
len('/content/drive/My Drive/Flickr_Data/features2.pkl')

49

In [ ]:
import string
def load_doc(filename):
  file=open(filename,'r')
  text=file.read()
  file.close()
  return text

def load_descriptions(doc):
  mapping=dict()
  for line in doc.split('\n'):
    tokens=line.split()
    if len(line) <2:
      continue
    image_id,image_desc = tokens[0], tokens[1:]
    image_id=image_id.split('.')[0]
    image_desc=' '.join(image_desc)

    if image_id not in mapping:
      mapping[image_id]= list()
    mapping[image_id].append(image_desc)

  return mapping


def clean_descriptions(descriptions):
  table=str.maketrans('','',string.punctuation)
  for key, desc_list in descriptions.items():
    for i in range(len(desc_list)):
      desc=desc_list[i]

      desc=desc.split()
      desc=[word.lower() for word in desc]
      desc=[w.translate(table) for w in desc]
      desc=[word for word in desc if len(word)>1]
      desc=[word for word in desc if word.isalpha()]
      desc_list[i]= ' '.join(desc)

def to_vocabulary(descriptions):
  all_desc=set()
  for key in descriptions.keys():
    [all_desc.update(d.split()) for d in descriptions [key ]]
  return all_desc

def save_descriptions(descriptions, filename):
  lines=list()
  for key ,desc_list in descriptions.items():
    for desc in desc_list:
      lines.append(key+ ' '+desc)
  data= '\n'.join(lines)
  file=open(filename,'w')
  file.write(data)
  file.close()

filename='/content/drive/My Drive/Flickr_Data/Flickr_TextData/Flickr8k.token.txt'
doc=load_doc(filename)
descriptions=load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))

clean_descriptions(descriptions)

vocabulary= to_vocabulary(descriptions)
print('vocabualry size: %d ' % len(vocabulary))

save_descriptions(descriptions,'/content/drive/My Drive/Flickr_Data/descriptions.txt' )



Loaded: 8092 
vocabualry size: 8763 


In [ ]:
from pickle import load
def load_doc(filename):
  file=open(filename,'r')
  text=file.read()
  file.close()
  return text

def load_set(filename):
  doc=load_doc(filename)
  dataset=list()
  for line in doc.split('\n'):
    if len(line)<1:
      continue
    identifier = line.split('.')[0]
    dataset.append(identifier)
  return set(dataset)


def load_clean_descriptions(filename, dataset):
  doc=load_doc(filename)
  descriptions=dict()
  for line in doc.split('\n'):
    tokens=line.split()
    image_id, image_desc=tokens[0], tokens[1:]
    if image_id in dataset:
      if image_id not in descriptions:
        descriptions[image_id]=list()
      desc='starseq ' + ' '.join(image_desc) +' endseq'
      descriptions[image_id].append(desc)
  return descriptions

def load_photo_features(filename, dataset):
  all_features= load(open(filename, 'rb'))
  features={k: all_features[k] for k in dataset}
  return features





In [ ]:

filename='/content/drive/My Drive/Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt'
train=load_set(filename)
print('Dataset: %d' % len(train))
train_descriptions=load_clean_descriptions('/content/drive/My Drive/Flickr_Data/descriptions.txt',train)
print('Descriptions: train=%d' % len(train_descriptions))

Dataset: 6000
Descriptions: train=6000


In [ ]:
train_features=load_photo_features('/content/drive/My Drive/Flickr_Data/features.pkl',train)
print('photos train=%d' % len(train_features))

photos train=6000


In [ ]:
type(train)

set

In [ ]:
train_descriptions


{'1000268201_693b08cb0e': ['starseq child in pink dress is climbing up set of stairs in an entry way endseq',
  'starseq girl going into wooden building endseq',
  'starseq little girl climbing into wooden playhouse endseq',
  'starseq little girl climbing the stairs to her playhouse endseq',
  'starseq little girl in pink dress going into wooden cabin endseq'],
 '1001773457_577c3a7d70': ['starseq black dog and spotted dog are fighting endseq',
  'starseq black dog and tricolored dog playing with each other on the road endseq',
  'starseq black dog and white dog with brown spots are staring at each other in the street endseq',
  'starseq two dogs of different breeds looking at each other on the road endseq',
  'starseq two dogs on pavement moving toward each other endseq'],
 '1002674143_1b742ab4b8': ['starseq little girl covered in paint sits in front of painted rainbow with her hands in bowl endseq',
  'starseq little girl is sitting in front of large painted rainbow endseq',
  'stars

In [ ]:
from keras.preprocessing.text import Tokenizer
def to_lines(descriptions):
  all_desc=list()
  for key in descriptions.keys():
    [all_desc.append(d) for d in descriptions[key]]
  return all_desc

def create_tokenizer(descriptions):
  lines= to_lines(descriptions)
  tokenizer=Tokenizer()
  tokenizer.fit_on_texts(lines)
  return tokenizer

tokenizer=create_tokenizer(train_descriptions)
vocab_size=len(tokenizer.word_index)+1
print('vocabulary size: %d' % vocab_size)

def max_length(descriptions):
  lines=to_lines(descriptions)
  return max(len(d.split()) for d in lines)

def create_sequences(tokenizer, max_length, desc_list, photo):
  X1,X2, y=list(),list(),list()
  for desc in desc_list:
    seq=tokenizer.texts_to_sequences([desc])[0]
    for i in range (1,len(seq)):
        in_seq, out_seq= seq[:i], seq[i]

        in_seq=pad_sequences([in_seq], maxlen=max_length)[0]
        out_seq= to_categorical([out_seq], num_classes= vocab_size)[0]

        X1.append(photo)
        X2.append(in_seq)
        y.append(out_seq)

  return array(X1), array(X2), array(y)





vocabulary size: 7579


In [ ]:
from numpy import array
import tensorflow
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Dense,LSTM,Embedding,Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from numpy import argmax
from nltk.translate.bleu_score import corpus_bleu



In [ ]:
def define_model(vocab_size,max_length):
  inputs1=Input(shape=(4096,))
  fe1=Dropout(0.5)(inputs1)
  fe2=Dense(256, activation='relu')(fe1)
  #sequence model#
  inputs2=Input(shape=(max_length,))
  se1=Embedding(vocab_size, 256, mask_zero=True)(inputs2)
  se2=Dropout(0.5)(se1)
  se3=LSTM(256)(se2)

  #decodel model#
  decoder1=add([fe2,se3])
  decoder2=Dense(256, activation='relu')(decoder1)
  outputs=Dense(vocab_size, activation='softmax')(decoder2)

  model=Model(inputs=[inputs1,inputs2], outputs=outputs)
  model.compile(loss='categorical_crossentropy', optimizer='adam')
  print(model.summary())
  return model
  

In [ ]:
def data_generator(descriptions, photos, tokenizer, max_length):
  while 1:
    for key, desc_list in descriptions.items():
      photo=photos[key][0]
      in_img, in_seq, out_word=create_sequences(tokenizer, max_length, desc_list, photo)
      yield [[in_img, in_seq], out_word]
      



In [ ]:
filename='/content/drive/My Drive/Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt'
train=load_set(filename)
print('dataset %d'% len(train))
train_descriptions=load_clean_descriptions('/content/drive/My Drive/Flickr_Data/descriptions.txt', train)
print('descriptions :train=%d' % len(train_descriptions))
train_features=load_photo_features('/content/drive/My Drive/Flickr_Data/features.pkl', train)
print('photos :train=%d'  % len(train_features))
tokenizer=create_tokenizer(train_descriptions)
vocab_size=len(tokenizer.word_index)+ 1
print('vocabulary size : %d' % vocab_size)
max_length=max_length(train_descriptions)
print('desriptions length: %d' % max_length)



dataset 6000
descriptions :train=6000
photos :train=6000
vocabulary size : 7579
desriptions length: 34


In [ ]:
model=define_model(vocab_size,max_length)
epochs=10
steps=len(train_descriptions)
for i in range(epochs):
  generator=data_generator(train_descriptions,train_features,tokenizer,max_length)
  model.fit_generator(generator,epochs=1, steps_per_epoch=steps, verbose=1)
  model.save('model_'+ str(i)+'.h5')

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 34)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 4096)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 34, 256)      1940224     input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 4096)         0           input_1[0][0]                    
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
6000/6000 [==============================] - 913s 152ms/step - loss: 4.6608
Epoch 1/1
6000/6000 [==============================] - 907s 151ms/step - loss: 3.8956
Epoch 1/1
6000/6000 [==============================] - 909s 152ms/step - loss: 3.6470
Epoch 1/1
6000/6000 [==============================] - 904s 151ms/step - loss: 3.4862
Epoch 1/1
6000/6000 [==============================] - 902s 150ms/step - loss: 3.3829
Epoch 1/1
6000/6000 [==============================] - 914s 152ms/step - loss: 3.3078
Epoch 1/1
6000/6000 [==============================] - 907s 151ms/step - loss: 3.2543
Epoch 1/1
6000/6000 [==============================] - 906s 151ms/step - loss: 3.2126
Epoch 1/1
6000/6000 [==============================] - 911s 152ms/step - loss: 3.1835
Epoch 1/1
6000/6000 [==============================] - 914s 152ms/step - loss: 3.1625


In [ ]:
def word_for_id(integer,tokenizer):
  for word,index in tokenizer.word_index.items():
    if index==integer:
      return word
  return None

def generate_desc(model,tokenizer, photo, max_length):
  in_text='starseq'
  for i in range(max_length):
    sequence=tokenizer.texts_to_sequences([in_text])[0]
    sequence= pad_sequences([sequence], maxlen=max_length)
    yhat=model.predict([photo,sequence], verbose=0)
    yhat=argmax(yhat)
    word=word_for_id(yhat,tokenizer)
    if word is None:
      break
    in_text+= ' ' + word
    if word=='endseq':
      break
  return in_text




In [ ]:
def evaluate_model(model, descriptions, photos, tokenizer,max_length):
  actual,predicted=list(),list()
  for key, desc_list in descriptions.items():
    yhat=generate_desc(model, tokenizer, photos[key], max_length)
    references=[d.split() for d in desc_list]
    actual.append(references)
    predicted.append(yhat.split())
  print('BLEU -1: %f' % corpus_bleu(actual,predicted, weights=(1.0,0,0,0)))
  print('BLEU -2: %f' % corpus_bleu(actual,predicted, weights=(0.5,0.5,0,0)))
  print('BLEU -3: %f' % corpus_bleu(actual,predicted, weights=(0.3,0.3,0.3,0)))
  print('BLEU -4: %f' % corpus_bleu(actual,predicted, weights=(0.25,0.25,0.25,0.25)))




In [ ]:
filename='/content/drive/My Drive/Flickr_Data/Flickr_TextData/Flickr_8k.testImages.txt'
test=load_set(filename)
print('Dataset: %d' % len(test))

test_descriptions=load_clean_descriptions('/content/drive/My Drive/Flickr_Data/descriptions.txt', test)
print('descriptions: test=%d' % len(test_descriptions))

test_features=load_photo_features('/content/drive/My Drive/Flickr_Data/features.pkl', test)
print('photos: test=%d' % len(test_features))


Dataset: 1000
descriptions: test=1000
photos: test=1000


In [ ]:
filename='model_9.h5'
model=load_model(filename)
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


BLEU -1: 0.409282
BLEU -2: 0.224118
BLEU -3: 0.154063
BLEU -4: 0.066090


In [ ]:
photo=extract_features('/content/drive/My Drive/Flickr_Data/Flicker8k_Dataset/3286822339_5535af6b93.jpg')
descriptions=generate_desc(model, tokenizer, photo, max_length)
print(descriptions)

starseq man in black shirt and jeans and black shirt and black shirt and black shirt and black shirt and black shirt and black shirt and black shirt and black shirt and black shirt and


In [ ]:


def extract_features(filename):
  model=VGG16()
  model.layers.pop()
  model=Model(inputs=model.inputs, outputs=model.layers[-1].output)
  image=load_img(filename, target_size=(224,224))
  image=img_to_array(image)
  image=image.reshape((1,image.shape[0], image.shape[1], image.shape[2]))
  image=preprocess_input(image)
  features= model.predict(image,verbose=0)
  return features
  print(model.summary())
  
max_length=34